Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

In [ ]:
#will need to split into training and testing/validation data - for the HS data, split randomly (70% training, 30% testing)

features.loc[features.LADnm != 'Nottingham', 'LADnm'] = 'Training'
features.loc[features.LADnm == 'Nottingham', 'LADnm'] = 'Test'
#features[features['LADnm'] == 'Training']

features_weighted.loc[features_weighted.LADnm != 'Nottingham', 'LADnm'] = 'Training'
features_weighted.loc[features_weighted.LADnm == 'Nottingham', 'LADnm'] = 'Test'

train = features[features['LADnm'] == 'Training']
test = features[features['LADnm'] == 'Test']

train_weighted = features_weighted[features_weighted['LADnm'] == 'Training']
test_weighted = features_weighted[features_weighted['LADnm'] == 'Test']

test_weighted.head()

x = features[features.columns[15:22]]
x = x.merge(features[["LADnm", "Employment"]], how = "inner", on = "Employment")
y = features[features.columns[1:3]]

x_train = train[train.columns[15:22]]
y_train = train[train.columns[1]]

x_test = test[test.columns[15:22]]
y_test = test[test.columns[1]]

print(x_train.head())
print("Number of Datapoints:", x_train.shape[0])
print("Number of Input Features:", x_train.shape[1])

x_weighted = features_weighted[features_weighted.columns[4:11]]
x_weighted = x_weighted.merge(features[["LADnm", "Employment"]], how = "inner", on = "Employment")
y_weighted = features_weighted[features_weighted.columns[1:3]]

x_train_weighted = train_weighted[train_weighted.columns[4:11]]
y_train_weighted = train_weighted[train_weighted.columns[1]]

x_test_weighted = test_weighted[test_weighted.columns[4:11]]
y_test_weighted = test_weighted[test_weighted.columns[1]]

print("Number of Datapoints:", x_train_weighted.shape[0])
print("Number of Input Features:", x_train_weighted.shape[1])

print(x_test_weighted.head())

In [ ]:
# import random forest regression from sklearn
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

rf = RandomForestRegressor(max_depth=2, random_state=42)

# fit to the training data
rf.fit(x_train, y_train)
print(rf.score(x_train, y_train))
# predict using the model
rf_train_pred = rf.predict(x_train)
rf_pred = rf.predict(x_test)

# append to the results dataframe
model_results = model_results.assign(rf_pred = rf_pred)
print(model_results)

# test the performance of the model
scores = cross_val_score(rf, x_train, y_train, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# repeat the above process with the weighted data
rf.fit(x_train_weighted, y_train_weighted)
print(rf.score(x_train_weighted, y_train_weighted))
rf_train_pred = rf.predict(x_train_weighted)
rf_pred_weighted = rf.predict(x_test_weighted)

model_results = model_results.assign(rf_pred_weighted = rf_pred_weighted)
print(model_results)

scores = cross_val_score(rf, x_train_weighted, y_train_weighted, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [1]:
# https://gis.stackexchange.com/questions/361318/create-pandas-dataframe-from-raster-image-one-row-per-pixel-with-bands-as-colu

# import rasterio as rio
# cross_lochs_test = rio.open('../../preprocessed_data/dimensionality_reduction/cross_lochs.tif')
# # number of bands
# cross_lochs_test.count
# # read bands
# array = cross_lochs_test.read()
# # convert to a DataFrame
# import pandas as pd
# cross_lochs_df = pd.DataFrame()
# cross_lochs_df['band1'] = array[0].ravel() 
# cross_lochs_df['band2'] = array[1].ravel() 
# cross_lochs_df['band3'] = array[2].ravel()
# cross_lochs_df.head(2)


,band1,band2,band3
0,0.003693,0.002533,0.002079
1,0.004860,0.001867,0.003720


In [9]:
cross_lochs_df.tail(2) # last

,band1,band2,band3
41404,0.003819,0.005305,0.008166
41405,0.003836,0.005264,0.007810


Want to iterate through the data rather than adding manually (358 bands)

In [2]:
import rasterio as rio
cross_lochs_test = rio.open('../../preprocessed_data/dimensionality_reduction/cross_lochs.tif')
# number of bands
cross_lochs_test.count
# read bands
array = cross_lochs_test.read()
# convert to a DataFrame
import pandas as pd
cross_lochs_df = pd.DataFrame()

array_num = 0
band_num = 1

for num in range(358):
    cross_lochs_df['band' +str(band_num)]=array[array_num].ravel()
    array_num += 1
    band_num += 1


cross_lochs_df.head(2)

# NB when do this, will want a PFT column, which will include in randmisation, then split the train and 
# test data into x (bands) and y (PFT)

/tmp/ipykernel_5361/878720774.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cross_lochs_df['band' +str(band_num)]=array[array_num].ravel()


,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
0,0.003693,0.002533,0.002079,0.003926,0.005210,0.007657,0.009435,0.012086,0.013435,0.014549,...,0.091244,0.089460,0.086190,0.089189,0.085616,0.086009,0.087522,0.086040,0.083325,0.084438
1,0.004860,0.001867,0.003720,0.005022,0.005861,0.008893,0.011048,0.012524,0.014608,0.017364,...,0.081579,0.082929,0.080569,0.080958,0.080036,0.079593,0.078432,0.079404,0.079279,0.079257


In [15]:
cross_lochs_df.tail(2)

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
41404,0.003819,0.005305,0.008166,0.010267,0.012013,0.015443,0.018905,0.021865,0.025044,0.028315,...,0.137687,0.135257,0.13593,0.136691,0.139425,0.138277,0.139343,0.138629,0.134105,0.136583
41405,0.003836,0.005264,0.007810,0.010020,0.011569,0.015417,0.018613,0.021824,0.024962,0.028246,...,0.136338,0.136514,0.13397,0.138620,0.136225,0.140556,0.142742,0.138437,0.135536,0.133259


In [17]:
cross_lochs_df.sample(frac=1)

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
40287,0.000511,0.001655,0.003924,0.005402,0.007443,0.009077,0.012082,0.014630,0.016577,0.017633,...,0.107732,0.107225,0.106756,0.107408,0.107252,0.107411,0.104268,0.108908,0.108616,0.107683
9114,0.000406,0.001715,0.004033,0.005382,0.006453,0.008712,0.010543,0.011861,0.014241,0.015767,...,0.061888,0.062016,0.061197,0.062315,0.060476,0.061022,0.063841,0.060661,0.062223,0.062948
18767,0.000653,0.001610,0.003916,0.005298,0.006453,0.009020,0.011253,0.012921,0.015189,0.017463,...,0.095936,0.098366,0.095529,0.098729,0.097210,0.096922,0.095636,0.098426,0.096515,0.097044
36073,0.000353,0.001810,0.003973,0.005393,0.006217,0.008229,0.009643,0.010650,0.013550,0.014747,...,0.048591,0.048514,0.049089,0.049185,0.045892,0.048974,0.047381,0.048703,0.050155,0.049107
16362,0.000840,0.001671,0.004474,0.005820,0.006877,0.009178,0.011383,0.013443,0.015870,0.018352,...,0.098849,0.098462,0.094971,0.098897,0.096815,0.097206,0.097553,0.100873,0.094470,0.094572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,0.000290,0.001052,0.002508,0.003482,0.004561,0.005603,0.007699,0.009844,0.011127,0.013416,...,0.063715,0.062346,0.061595,0.063027,0.060683,0.061418,0.062388,0.061216,0.060151,0.060931
25206,0.000707,0.001725,0.004237,0.005524,0.007011,0.009181,0.011218,0.013006,0.015352,0.017291,...,0.087913,0.087922,0.085807,0.086542,0.085017,0.085695,0.088290,0.088153,0.083179,0.087826
23716,0.002064,0.003063,0.005019,0.006502,0.007831,0.010444,0.013017,0.015212,0.017911,0.020388,...,0.101365,0.098933,0.098180,0.100190,0.098743,0.097916,0.098196,0.099386,0.096835,0.100246
26673,0.001290,0.002716,0.004745,0.006297,0.007996,0.010215,0.012544,0.015220,0.017086,0.018660,...,0.041404,0.039811,0.040037,0.040380,0.039581,0.039644,0.040446,0.038736,0.040796,0.038197


In [18]:
percentage = round(len(cross_lochs_df)/100*70) 
train = cross_lochs_df.head(percentage)  
test = cross_lochs_df.iloc[percentage:len(df),:]

In [22]:
train.describe()

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
count,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,...,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000,28984.000000
mean,0.001134,0.001992,0.003822,0.004933,0.005949,0.008158,0.010177,0.011870,0.013993,0.015969,...,0.077656,0.077673,0.076666,0.077966,0.076429,0.076632,0.077051,0.077557,0.076219,0.076216
std,0.000925,0.000795,0.001088,0.001392,0.001667,0.002213,0.002603,0.002927,0.003304,0.003708,...,0.032486,0.032840,0.032430,0.032524,0.032104,0.032112,0.031907,0.032501,0.032164,0.032386
min,-0.002965,-0.002399,-0.001051,-0.000417,0.000495,0.001153,0.002203,0.002932,0.004199,0.005276,...,0.002212,0.001731,0.001499,0.000951,0.001166,0.000971,0.000995,0.001656,0.001594,0.000821
25%,0.000402,0.001546,0.003422,0.004474,0.005407,0.007536,0.009432,0.011022,0.013051,0.014923,...,0.068567,0.068141,0.066992,0.067786,0.066356,0.066221,0.066316,0.066525,0.065091,0.064740
50%,0.000885,0.001831,0.003988,0.005248,0.006376,0.008788,0.010914,0.012682,0.014930,0.017026,...,0.087392,0.087547,0.086369,0.087627,0.085864,0.085994,0.086389,0.087117,0.085568,0.085514
75%,0.001791,0.002562,0.004485,0.005779,0.006961,0.009493,0.011762,0.013654,0.016015,0.018239,...,0.099628,0.099968,0.098717,0.100250,0.098558,0.098806,0.099082,0.100127,0.098588,0.098778
max,0.006699,0.005923,0.008701,0.010647,0.012477,0.015521,0.018094,0.021029,0.023207,0.026678,...,0.144066,0.146195,0.146343,0.147946,0.143685,0.148298,0.144379,0.146654,0.145820,0.145754


In [23]:
test.describe()

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
count,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,...,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000,12422.000000
mean,0.001090,0.002197,0.004481,0.005944,0.007176,0.009944,0.012325,0.014390,0.016882,0.019266,...,0.097096,0.097078,0.095952,0.097733,0.095914,0.096258,0.096746,0.097334,0.095678,0.095795
std,0.001047,0.001243,0.001429,0.001804,0.002086,0.002436,0.002749,0.003123,0.003455,0.003784,...,0.021977,0.022096,0.021871,0.022085,0.021968,0.022153,0.022035,0.022279,0.021959,0.022150
min,-0.002143,-0.002133,-0.000860,-0.000089,-0.000249,0.001377,0.002358,0.003983,0.005285,0.006615,...,0.005717,0.005417,0.004336,0.005099,0.006427,0.006406,0.005006,0.005507,0.003539,0.003074
25%,0.000367,0.001221,0.003357,0.004425,0.005407,0.007931,0.010115,0.011955,0.014209,0.016463,...,0.089629,0.089929,0.088754,0.090096,0.088178,0.088439,0.088824,0.089354,0.088067,0.087992
50%,0.000548,0.001882,0.004385,0.005980,0.007240,0.009842,0.012224,0.014143,0.016653,0.018931,...,0.101244,0.101438,0.100179,0.101799,0.100055,0.100324,0.100787,0.101720,0.099996,0.100157
75%,0.001814,0.003227,0.005548,0.007295,0.008756,0.011776,0.014354,0.016682,0.019403,0.021995,...,0.110716,0.110692,0.109514,0.111487,0.109613,0.110011,0.110421,0.111132,0.109478,0.109828
max,0.004363,0.005568,0.008453,0.010667,0.012651,0.016496,0.019949,0.023107,0.026461,0.029830,...,0.153268,0.154181,0.149297,0.155180,0.150108,0.150619,0.150237,0.155809,0.150343,0.153459


In [24]:
print(28984/(28984+12422)*100)

69.99951697821571


Then need to split into X (all the bands) and y (PFT)

When compelte, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

print(clf.predict([x_test]))

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html 
# - look at for parameters and additional functions

# From FBA: (more detail at start of page)

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

# rf = RandomForestRegressor(max_depth=2, random_state=42)

# # fit to the training data
# rf.fit(x_train, y_train)
# print(rf.score(x_train, y_train))
# # predict using the model
# rf_train_pred = rf.predict(x_train)
# rf_pred = rf.predict(x_test)

# # append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# # test the performance of the model
# scores = cross_val_score(rf, x_train, y_train, cv=3)
# print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))